<a href="https://colab.research.google.com/github/DIPANJAN001/Andrew-Ng-Machine-Learning-Notes/blob/master/last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Concatenate, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.losses import contrastive_loss
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Load your one-hot encoded dataset (categorical features)
# Ensure you have pairs of examples (severe, non-severe) for training

# Create pairs for the Siamese network
positive_pairs = []  # Pairs of similar claims (severe vs. severe)
negative_pairs = []  # Pairs of dissimilar claims (severe vs. non-severe)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(np.concatenate((positive_pairs, negative_pairs)), np.concatenate((np.ones(len(positive_pairs)), np.zeros(len(negative_pairs))), test_size=0.2, random_state=42)

# Define the Siamese network architecture
def siamese_network(input_shape):
    input = Input(shape=input_shape)
    x = Dense(128, activation='relu')(input)  # Adjust units and activation as needed
    x = Dense(64, activation='relu')(x)  # Add more layers or customize as needed
    output = Dense(32)(x)  # Embedding dimension

    return Model(input, output)

# Define the contrastive loss function
def contrastive_loss_with_margin(margin=1.0):
    def contrastive_loss(y_true, y_pred):
        square_pred = tf.square(y_pred)
        margin_square = tf.square(tf.maximum(margin - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

    return contrastive_loss

# Build and compile the Siamese network
input_shape = (input_dimension,)  # Adjust input dimension
siamese_model = siamese_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

output_a = siamese_model(input_a)
output_b = siamese_model(input_b)

# Use Lambda layer to calculate the Euclidean distance between the embeddings
distance = Lambda(lambda x: tf.norm(x[0] - x[1], axis=-1, keepdims=True), output_shape=(1,))(Concatenate()([output_a, output_b]))

siamese_model = Model(inputs=[input_a, input_b], outputs=distance)

optimizer = Adam(learning_rate=0.001)
siamese_model.compile(loss=contrastive_loss_with_margin(margin=1.0), optimizer=optimizer)

# Train the Siamese network
siamese_model.fit([X_train[:, 0], X_train[:, 1]], y_train, batch_size=64, epochs=10, validation_split=0.2)

# Define a single-claim classifier
single_claim_input = Input(shape=input_shape)

# Add more Dense layers as needed
x = Dense(128, activation='relu')(single_claim_input)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

single_claim_classifier = Model(single_claim_input, output)

# Compile the single-claim classifier
single_claim_classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Extract embeddings for a single test claim using the Siamese network
test_claim_input = ...  # Your one-hot encoded test claim (preprocessed and shaped as needed)
test_claim_embedding = siamese_model.predict([test_claim_input, test_claim_input])

# Pass the test claim embedding to the single-claim classifier
severe_probability = single_claim_classifier.predict(test_claim_embedding)

# 'severe_probability' will contain the probability of the test claim being "severe."


In [ ]:
# Extract embeddings for a single test claim using the Siamese network
test_claim_input = ...  # Your one-hot encoded test claim (preprocessed and shaped as needed)

# Use the Siamese model to get the embedding for the single test claim
test_claim_embedding = siamese_model.predict([test_claim_input, test_claim_input])

# Pass the test claim embedding to the single-claim classifier
severe_probability = single_claim_classifier.predict(test_claim_embedding)

# 'severe_probability' will contain the probability of the test claim being "severe."


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Assume one_hot_data is your one-hot encoded feature matrix and labels is your 0/1 labels.
one_hot_data = ...
labels = ...

# Create pairs and labels for Siamese network training
positive_pairs = []
negative_pairs = []

# Loop through the data and create pairs
for i in range(len(one_hot_data)):
    for j in range(len(one_hot_data)):
        if labels[i] == labels[j]:
            positive_pairs.append((one_hot_data[i], one_hot_data[j], 1))
        else:
            negative_pairs.append((one_hot_data[i], one_hot_data[j], 0))

# Combine positive and negative pairs
pairs = positive_pairs + negative_pairs
labels = np.array([pair[2] for pair in pairs])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(pairs, labels, test_size=0.2, random_state=42)


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
import numpy as np

# Define a custom F1 scorer for GridSearchCV
custom_f1_scorer = make_scorer(f1_score)

# Create a parameter grid to search through
param_grid = {
    'num_leaves': [31, 50, 100],
    'learning_rate': [0.05, 0.1, 0.2],
    'min_child_samples': [20, 50, 100],
    # Add other hyperparameters you want to tune here
}

# Load and preprocess your dataset
# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Calculate class weights
scale_pos_weight = len(y_train) / (2 * np.bincount(y_train))

# Create the base LightGBM model
base_model = lgb.LGBMClassifier(scale_pos_weight=scale_pos_weight, objective='binary')

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, scoring=custom_f1_scorer, cv=5)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and best F1 score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

# Get the best model with the optimal hyperparameters
best_model = grid_search.best_estimator_

# Predict probabilities and make binary predictions using the best model
y_pred_prob = best_model.predict_proba(X_test)[:, 1]
threshold = 0.5  # Adjust this threshold as needed to balance TP rate and precision
y_pred = (y_pred_prob > threshold).astype(int)

# Calculate the custom F1 score
custom_f1 = f1_score(y_test, y_pred)
print("Custom F1 Score (Best Model):", custom_f1)


In [ ]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer
import numpy as np

# Load and preprocess your dataset
# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define a custom metric that balances false negatives and false positives
def custom_balanced_f1(y_true, y_pred):
    f1 = f1_score(y_true, y_pred)
    fn = np.sum((y_true == 1) & (y_pred == 0))  # False Negatives
    fp = np.sum((y_true == 0) & (y_pred == 1))  # False Positives
    balanced_f1 = f1 - 0.5 * fn - 0.5 * fp  # Add penalties for false negatives and false positives
    return balanced_f1

# Create a scorer from the custom metric
custom_scorer = make_scorer(custom_balanced_f1)

# Hyperparameter tuning for XGBoost with the custom metric
xgb_param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300]
}

xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
xgb_grid_search = GridSearchCV(xgb_model, param_grid=xgb_param_grid, scoring=custom_scorer, cv=5)
xgb_grid_search.fit(X_train, y_train)
best_xgb_model = xgb_grid_search.best_estimator_

# Hyperparameter tuning for LightGBM with the custom metric
lgb_param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300]
}

lgb_model = lgb.LGBMClassifier(objective='binary', random_state=42)
lgb_grid_search = GridSearchCV(lgb_model, param_grid=lgb_param_grid, scoring=custom_scorer, cv=5)
lgb_grid_search.fit(X_train, y_train)
best_lgb_model = lgb_grid_search.best_estimator_

# Train the best XGBoost and LightGBM models
best_xgb_model.fit(X_train, y_train)
best_lgb_model.fit(X_train, y_train)

# Generate predictions from both models
xgb_predictions = best_xgb_model.predict(X_test)
lgb_predictions = best_lgb_model.predict(X_test)

# Combine predictions (simple averaging)
ensemble_predictions = (xgb_predictions + lgb_predictions) / 2

# Evaluate the ensemble's performance using the custom metric
ensemble_custom_f1 = custom_balanced_f1(y_test, ensemble_predictions)

print("Ensemble Custom Balanced F1 Score:", ensemble_custom_f1)
